# 식품영양성분_with_tags.csv => Pinecone에 적재 후 LLM 호출

In [2]:
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI 

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [11]:
assert OPENAI_API_KEY and PINECONE_API_KEY, "API 키를 .env에 설정하세요."

# OpenAI / Pinecone 클라이언트
oai = OpenAI(api_key=OPENAI_API_KEY)
pc  = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME = "food-test1"   # 인덱스 이름
NAMESPACE  = "foods-ns1"    # namespace (선택)

# 인덱스 생성(없으면)
try:
    # v5 SDK: .names() 지원
    names = pc.list_indexes().names()
except Exception:
    # 구버전 호환
    names = [x.name for x in pc.list_indexes()]

if INDEX_NAME not in names:
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,             # text-embedding-3-small 차원
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(INDEX_NAME)

# 전처리 완료 CSV 경로(파일명 꼭 확인!)
CSV_PATH = "식품영양성분_with_tags.csv"  # ← 네가 마지막에 만든 파일명으로
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
print(f"🍱 총 {len(df)}개 로드 / 컬럼 예시: {list(df.columns)[:10]}")

🍱 총 548개 로드 / 컬럼 예시: ['food_id', 'food_name', 'ref_amount', 'energy_kcal', 'protein_g', 'fat_g', 'carb_g', 'sugar_g', 'sodium_mg', 'chol_mg']


In [9]:
def make_text(row):
    return f"""
    음식명: {row['food_name']} ({row['ref_amount']}g 기준)
    칼로리: {row['energy_kcal']}kcal
    단백질: {row['protein_g']}g, 지방: {row['fat_g']}g, 탄수화물: {row['carb_g']}g, 당: {row['sugar_g']}g
    나트륨: {row['sodium_mg']}mg, 콜레스테롤: {row['chol_mg']}mg
    포화지방: {row['sat_fat_g']}g, 트랜스지방: {row['trans_fat_g']}g
    중량: {row['food_weight']}g
    """.strip()
# ----------------------------
#  메타데이터 구성 함수 수정 (영양 정보 추가)
# ----------------------------
def make_metadata(row):
    return {
        "norm_tags": row.get("norm_tags", ""),
        "display_tags": row.get("display_tags", ""),
        "content": row.get("content", ""),
        "food_name": row.get("food_name", ""),
        "energy_kcal": row.get("energy_kcal", 0.0),
        "protein_g": row.get("protein_g", 0.0),
        "fat_g": row.get("fat_g", 0.0),
        "carb_g": row.get("carb_g", 0.0),
        "food_weight": row.get("food_weight", 0.0),
    }


In [10]:
# ----------------------------
#  Pinecone 배치 업로드
# ----------------------------
client = oai
batch_size = 100  # 🚀 한번에 100개씩 임베딩 + 업로드
for i in tqdm(range(0, len(df), batch_size), desc="🍽️ Pinecone 업로드 중"):
    batch = df.iloc[i:i + batch_size]

    # 1. 텍스트 리스트 생성
    batch_texts = [make_text(row) for _, row in batch.iterrows()]

    # 2. OpenAI에 배치 임베딩 요청 (한 번에 100개)
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=batch_texts
    )
    embeddings = [d.embedding for d in response.data]

    # 3. Pinecone 벡터 리스트 구성
    batch_vectors = []
    for (idx, row), emb in zip(batch.iterrows(), embeddings):
        meta = make_metadata(row)
        batch_vectors.append({
            "id": str(row["food_id"]),
            "values": emb,
            "metadata": meta
        })

    # 4. Pinecone 업서트 (namespace 지정)
    index.upsert(vectors=batch_vectors, namespace=NAMESPACE)

    print(f"  → {i + len(batch_vectors)}/{len(df)} 업로드 완료")

print(f"\n✅ Pinecone 업로드 완료! ({len(df)}개 항목, namespace='{NAMESPACE}')")

🍽️ Pinecone 업로드 중:  17%|█▋        | 1/6 [00:16<01:20, 16.13s/it]

  → 100/548 업로드 완료


🍽️ Pinecone 업로드 중:  33%|███▎      | 2/6 [00:20<00:36,  9.23s/it]

  → 200/548 업로드 완료


🍽️ Pinecone 업로드 중:  50%|█████     | 3/6 [00:25<00:21,  7.11s/it]

...

  → 400/548 업로드 완료


🍽️ Pinecone 업로드 중:  83%|████████▎ | 5/6 [00:35<00:06,  6.10s/it]

  → 500/548 업로드 완료


🍽️ Pinecone 업로드 중: 100%|██████████| 6/6 [00:38<00:00,  6.36s/it]

  → 548/548 업로드 완료

✅ Pinecone 업로드 완료! (548개 항목, namespace='foods-ns1')


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7, 
    max_tokens=1024, 
    openai_api_key=OPENAI_API_KEY,
)

### 아래 두 셀은 실행 X -> 바로 수정된 프롬프트로 넘어가기

In [ ]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")

prompt_template = f"""
당신은 최고 수준의 영양사이며, 사용자 요청 조건과 한국인의 일반적인 식습관을 고려하여 식단을 구성합니다.

**핵심 규칙:**
1. 출력은 오직 <출력 예시>의 **형태와 패턴**을 **한 글자도 틀리지 않고** 엄격히 준수해야 합니다.
2. 절대 마크다운(**, ## 등)이나 불필요한 구분선을 사용하지 마세요.
3. [음식의 조화]: 각 후보는 **최소 3가지 이상의 음식**을 포함하는 조화로운 조합이어야 합니다.
4. [칼로리]: 아침, 점심, 저녁의 총합 칼로리가 1800kcal ~ 2500kcal 사이에 오도록 스스로 계산을 완료하세요. "조정 필요" 문구는 절대 금지합니다.
5. 절대 간식을 추가하지 마세요. 모든 음식은 <참고 문서>에서 찾은 음식만 사용하세요. 브랜드명은 제거하세요.

<참고 문서> (음식명, 총 칼로리, 영양 정보 등)
{{context}}

<사용자 요청>
{{query}}

<출력 예시> (반드시 이 형식의 텍스트 패턴을 따르세요. 실제 날짜와 칼로리를 계산하여 대체하세요.)
< {today} 식단 추천 >
아침
후보1 : 현미밥 + 닭가슴살 + 시금치나물 (총 550 kcal)
후보2 : 오트밀 + 삶은 계란 + 저지방우유 (총 480 kcal)
후보3 : 곤드레밥 + 소고기미역국 + 깍두기 (총 650 kcal)
점심
후보1 : 잡곡밥 + 제육볶음 + 콩나물국 (총 800 kcal)
후보2 : 치킨텐더샐러드 + 통밀빵 + 요거트 (총 750 kcal)
후보3 : 김치찌개 + 두부부침 + 흑미밥 (총 780 kcal)
저녁
후보1 : 연어스테이크 + 구운채소 + 렌틸콩수프 (총 600 kcal)
후보2 : 현미밥 + 멸치볶음 + 미역국 + 달걀후라이 (총 630 kcal)
후보3 : 닭가슴살 볶음밥 + 단백질쉐이크 (총 650 kcal)
이유 : 근육 증진을 위해 고단백과 건강한 지방 위주로 구성하였으며, 나트륨을 낮게 유지하여 건강을 고려했습니다. 하루 총 칼로리는 [계산된 총합 칼로리]kcal 입니다.
"""

In [ ]:
# -----------------------------
# 3️⃣ Pinecone 검색 후 Context 구성 (LLM이 계산 및 조합하도록)
# -----------------------------
# 쿼리 예시
query = "근육증진형, 고단백, 고지방, 저나트륨 식단 중심으로, 아침/점심/저녁 세 끼만 추천해줘. 각 끼니별로 3가지의 완벽한 식사 조합을 만들고, 하루 총 칼로리가 1800~2500kcal 사이에 오도록 정확히 계산해줘."

query_emb = client.embeddings.create(
    model="text-embedding-3-small",
    input=query
).data[0].embedding

results = index.query(
    vector=query_emb,
    top_k=50,
    namespace="foods-ns1",
    include_metadata=True
)

# Context 구성 수정: 'food_weight' 키 제거
context = "\n".join([
    f"음식명: {m['metadata']['food_name']}, " 
    f"칼로리(총): {m['metadata']['energy_kcal']}kcal, "
    f"단백질: {m['metadata']['protein_g']}g, "
    f"지방: {m['metadata']['fat_g']}g, "
    f"탄수화물: {m['metadata']['carb_g']}g, " 
    f"태그: {m['metadata']['display_tags']}"
    for m in results["matches"]
])

# -----------------------------
# 4️⃣ LLM 실행 코드는 그대로 유지
# -----------------------------
chain = LLMChain(llm=llm, prompt=template)
response = chain.run({"context": context, "query": query})

print(" 추천 결과:")
print(response)

# 수정된 프롬프트

In [ ]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")

# 4) 프롬프트 템플릿
prompt_template_str = """
당신은 최고 수준의 영양사이며, 사용자 요청 조건과 한국인의 일반적인 식습관을 고려하여 식단을 구성합니다.

핵심 규칙:
1. 출력은 오직 <출력 예시>의 형태와 패턴을 한 글자도 틀리지 않고 엄격히 준수해야 합니다.
2. 절대 마크다운(**, ## 등)이나 불필요한 구분선을 사용하지 마세요.
3. [음식의 조화]: 각 후보는 최소 3가지 이상의 음식을 포함하는 조화로운 조합이어야 합니다.
4. [칼로리]: 아침, 점심, 저녁의 총합 칼로리가 1800kcal ~ 2500kcal 사이에 오도록 스스로 계산을 완료하세요. "조정 필요" 문구는 절대 금지합니다.
5. 모든 음식은 <참고 문서>에서 찾은 음식만 사용하세요. 브랜드명은 제거하세요.

<참고 문서> (음식명, 총 칼로리, 영양 정보 등)
{context}

<사용자 요청>
{query}

<출력 예시> (참고만 하세요. 예시 그대로 출력하지 마세요.)
< {today} 식단 추천 >
아침
후보1 : 현미밥 + 닭가슴살 + 시금치나물 (총 550 kcal)
후보2 : 오트밀 + 삶은 계란 + 저지방우유 (총 480 kcal)
후보3 : 곤드레밥 + 소고기미역국 + 깍두기 (총 650 kcal)
점심
후보1 : 잡곡밥 + 제육볶음 + 콩나물국 (총 800 kcal)
후보2 : 치킨텐더샐러드 + 통밀빵 + 요거트 (총 750 kcal)
후보3 : 김치찌개 + 두부부침 + 흑미밥 (총 780 kcal)
저녁
후보1 : 연어스테이크 + 구운채소 + 렌틸콩수프 (총 600 kcal)
후보2 : 현미밥 + 멸치볶음 + 미역국 + 달걀후라이 (총 630 kcal)
후보3 : 닭가슴살 볶음밥 + 단백질쉐이크 (총 650 kcal)

이유는 {today} 식단 추천을 모두 출력하면 가장 마지막에 출력하세요.
이유 : 근육 증진을 위해 고단백과 건강한 지방 위주로 구성하였으며, 나트륨을 낮게 유지하여 건강을 고려했습니다. 하루 총 칼로리는 [계산된 총합 칼로리]kcal 입니다.
""".strip()

template = PromptTemplate(
    template=prompt_template_str,
    input_variables=["context", "query", "today"]
)

In [13]:
client=oai
# 5) Pinecone 검색 → context 구성
user_query = "다이어트형, 고단백, 저지방, 저탄수 식단 중심으로, 아침/점심/저녁 세 끼를 추천해줘. 각 끼니별로 3가지의 완벽한 식사 조합을 만들고, 하루 총 칼로리가 1800~2500kcal 사이에 오도록 정확히 계산해줘. 오늘을 시작으로 총 7일간의 식단을 추천해줘."

# 임베딩 (OpenAI 클라이언트는 oai 사용)
qv = oai.embeddings.create(
    model="text-embedding-3-small",
    input=user_query
).data[0].embedding

# Pinecone 쿼리 (결과 표준화)
res = index.query(
    vector=qv,
    top_k=50,
    namespace=NAMESPACE,
    include_metadata=True
)
res = res.to_dict() if hasattr(res, "to_dict") else res
matches = res.get("matches", [])

# context 문자열 만들기 (결측 방어)
lines = []
for m in matches:
    md = m.get("metadata", {}) or {}
    name = str(md.get("food_name", "")).strip()
    kcal = md.get("energy_kcal", "")
    pro  = md.get("protein_g", "")
    fat  = md.get("fat_g", "")
    carb = md.get("carb_g", "")
    disp = md.get("display_tags", "")
    if name:  # 이름이 있는 것만
        lines.append(
            f"음식명: {name}, 칼로리(총): {kcal}kcal, 단백질: {pro}g, 지방: {fat}g, 탄수화물: {carb}g, 태그: {disp}"
        )
context = "\n".join(lines)

# 6) LLM 실행
chain = LLMChain(llm=llm, prompt=template)
response = chain.run({"context": context, "query": user_query, "today": today})

print("추천 결과:")
print(response)

추천 결과:
< 2025-10-17 식단 추천 >
아침  
후보1 : 달걀말이 + 달걀찜 + 동치미 (총 264 kcal)  
후보2 : 달걀국 + 달걀말이 + 오이김치 (총 217 kcal)  
후보3 : 달걀찜 + 달걀장조림 + 잔멸치꽈리고추볶음 (총 354 kcal)  
점심  
후보1 : 오징어볶음 + 돌솥비빔밥 + 당근볶음 (총 370 kcal)  
후보2 : 오징어덮밥 + 연근조림 + 오이김치 (총 322 kcal)  
후보3 : 날치알김밥 + 오징어튀김 + 동치미 (총 437 kcal)  
저녁  
후보1 : 오리탕 + 달걀찜 + 미역줄기볶음 (총 266 kcal)  
후보2 : 오징어조림 + 짬뽕밥 + 단무지 (총 203 kcal)  
후보3 : 치킨데리야끼 + 오이김치 + 미역국 (총 192 kcal)  
이유 : 다이어트형 및 고단백, 저지방, 저탄수 위주로 구성하였으며, 영양 균형을 고려했습니다. 하루 총 칼로리는 2180kcal 입니다.  

< 2025-10-18 식단 추천 >
아침  
후보1 : 달걀찜 + 달걀국 + 오이김치 (총 217 kcal)  
후보2 : 달걀말이 + 동치미 + 잔멸치꽈리고추볶음 (총 320 kcal)  
후보3 : 달걀말이 + 달걀장조림 + 미역줄기볶음 (총 359 kcal)  
점심  
후보1 : 오징어볶음 + 돌솥비빔밥 + 당근볶음 (총 370 kcal)  
후보2 : 오징어덮밥 + 연근조림 + 동치미 (총 322 kcal)  
후보3 : 날치알김밥 + 오징어튀김 + 미역국 (총 437 kcal)  
저녁  
후보1 : 오리탕 + 달걀찜 + 미역줄기볶음 (총 266 kcal)  
후보2 : 오징어조림 + 짬뽕밥 + 단무지 (총 203 kcal)  
후보3 : 치킨데리야끼 + 오이김치 + 미역국 (총 192 kcal)  
이유 : 다이어트형 및 고단백, 저지방, 저탄수 위주로 구성하였으며, 영양 균형을 고려했습니다. 하루 총 칼로리는 2180kcal 입니다.  

< 2025-10-19 식단 추천 >
아침  

# 주간 식단 생성 프롬프트

In [23]:
# (A) 날짜별 프롬프트 (이유 금지)
# === REPLACE: 일자별 프롬프트 ===
day_prompt_str = """
당신은 최고 수준의 영양사입니다. 아래 규칙과 참고 문서 내 음식만 사용하여 식단을 구성하세요.

핵심 규칙:
1) 출력 형식은 <출력 형식>을 정확히 따르세요. 마크다운/불필요한 문구/여분 줄 금지.
2) 각 후보는 최소 3가지 이상의 음식으로 ‘조합’하세요(단품 금지).
3) 하루 총칼로리(아침+점심+저녁 합계)는 {daily_target_kcal}kcal ±10% 내에 들도록 스스로 계산하세요.
4) 모든 음식은 <참고 문서>의 항목만 사용하고, 브랜드명 제거.

[다양성 규칙 — 아주 중요]
- 같은 끼니의 3개 후보는 서로 다른 **주요 단백질/주식**을 사용해야 합니다.
- 예시: 후보1이 '닭가슴살' 중심이면, 후보2는 '돼지고기'나 '콩류' 중심으로, 후보3은 '생선'이나 '계란' 중심으로 구성해야 합니다.
- **아침 후보 1, 2, 3은 모두 달걀/오트밀/죽 등 같은 주식 계열을 사용하면 안 됩니다.**
- 같은 후보 안에서도 가능한 한 서로 다른 카테고리(주식·단백질·야채·국/반찬)로 조합하세요.

[중간 요약 금지]
- 이 날짜 블록에서는 ‘이유/요약/주간’ 등 어떤 설명도 쓰지 마세요.
- ‘이유/요약/주간/총합’이라는 단어 자체를 출력하지 마세요.

<참고 문서>
{context}

<사용자 요청>
{query}

<출력 형식>
< {date} 식단 추천 >
아침
후보1 : 음식A + 음식B + 음식C (총 nnn kcal)
후보2 : 음식A + 음식B + 음식C (총 nnn kcal)
후보3 : 음식A + 음식B + 음식C (총 nnn kcal)
점심
후보1 : 음식A + 음식B + 음식C (총 nnn kcal)
후보2 : 음식A + 음식B + 음식C (총 nnn kcal)
후보3 : 음식A + 음식B + 음식C (총 nnn kcal)
저녁
후보1 : 음식A + 음식B + 음식C (총 nnn kcal)
후보2 : 음식A + 음식B + 음식C (총 nnn kcal)
후보3 : 음식A + 음식B + 음식C (총 nnn kcal)
""".strip()


In [24]:
import re

# 1) 같은 끼니에서 '계란/닭/소/돼지/생선/두부/밥/국/김치/빵/면/샐러드' 가족 중복 여부를 대략 체크
FAMILIES = {
    "계란": ["계란","달걀","스크램블","프라이","오믈렛","에그"],
    "닭": ["닭","치킨","닭가슴살","닭고기"],
    "소": ["소고기","쇠고기","우육","불고기","스테이크"],
    "돼지": ["돼지","제육","삼겹","갈비","돈","목살"],
    "생선": ["생선","고등어","연어","오징어","코다리","갈치","참치","회","초밥"],
    "두부콩": ["두부","두유","콩","렌틸","병아리콩","청국장","된장(콩)"],
    "밥곡류": ["밥","현미","잡곡","곤드레","죽","오트밀","보리","귀리","빵","토스트","통밀"],
    "국탕찌개": ["국","탕","찌개","수프","미역국","된장국","콩나물국"],
    "김치절임": ["김치","깍두기","동치미","무짠지","절임"],
    "샐러드야채": ["샐러드","야채","나물","볶음야채","무침"],
    "면류": ["면","우동","국수","짬뽕","라면","파스타"]
}

def detect_families(text):
    fams = set()
    for fam, keys in FAMILIES.items():
        for k in keys:
            if k in text:
                fams.add(fam); break
    return fams

def too_similar_three_lines(block_lines):
    """'후보1/2/3 :' 세 줄에서 가족 중복이 많은지 간단 체크"""
    cand_lines = [ln for ln in block_lines if ln.strip().startswith("후보")]
    fam_sets = [detect_families(ln) for ln in cand_lines]
    # 동일 가족 교집합이 크면(=전부 같은 축 섞임) 실패로 간주
    if len(cand_lines) == 3:
        union = set().union(*fam_sets)
        # 가족 다양성이 2 이하이면 너무 단조롭다고 판단
        return len(union) <= 2
    return False

# 2) 중간에 섞여 나온 '요약/주간/이유' 라인 제거
def sanitize_daily(text):
    lines = []
    for ln in text.splitlines():
        if re.search(r"(주간|요약|이유)\s*[:：]", ln):  # 금지 라벨 사전 제거
            continue
        lines.append(ln)
    return "\n".join(lines).strip()


In [25]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
import datetime

# LLM (필요시 gpt-4o-mini로 비용↓/길이↑)
llm_day = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, max_tokens=900, openai_api_key=OPENAI_API_KEY)
llm_sum = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, max_tokens=200, openai_api_key=OPENAI_API_KEY)

# 1) 사용자 쿼리(일일 2500kcal 목표 명시)
user_query = (
    "다이어트형, 고단백, 저지방, 저탄수 위주로 아침/점심/저녁 각 3가지 후보의 식사 조합을 추천해줘. "
    "하루 총 섭취 칼로리는 2500kcal 목표에 맞춰줘."
)
daily_target_kcal = "2500"

# 2) Pinecone에서 컨텍스트 수집
qv = oai.embeddings.create(model="text-embedding-3-small", input=user_query).data[0].embedding
res = index.query(vector=qv, top_k=80, namespace=NAMESPACE, include_metadata=True)
res = res.to_dict() if hasattr(res, "to_dict") else res
matches = res.get("matches", [])

# context 문자열 구성(결측 방어)
lines = []
for m in matches:
    md = m.get("metadata", {}) or {}
    name = str(md.get("food_name", "")).strip()
    kcal = md.get("energy_kcal", "")
    pro  = md.get("protein_g", "")
    fat  = md.get("fat_g", "")
    carb = md.get("carb_g", "")
    disp = md.get("display_tags", "")
    if name:
        lines.append(f"음식명: {name}, 칼로리(총): {kcal}kcal, 단백질: {pro}g, 지방: {fat}g, 탄수화물: {carb}g, 태그: {disp}")
context = "\n".join(lines)

# 3) 7일 날짜 생성
start = datetime.date.today()
dates = [(start + datetime.timedelta(days=i)).strftime("%Y-%m-%d") for i in range(7)]

# 4) 날짜별 생성(이유 없이 포맷만) + 다양성 검사/정리
day_chain = LLMChain(llm=llm_day, prompt=day_template)
daily_outputs = []
for d in dates:
    out = day_chain.run({
        "context": context,
        "query": user_query,
        "date": d,
        "daily_target_kcal": daily_target_kcal
    }).strip()

    out = sanitize_daily(out)

    # 끼니별 블록 분할 후 간단 다양성 검사, 실패 시 1회 재시도
    blocks = re.split(r"\n(?=점심|저녁)", out, maxsplit=2)  # "아침\n...점심\n...저녁\n..." 형태 가정
    needs_retry = any(too_similar_three_lines(b.splitlines()) for b in blocks)

    if needs_retry:
        # 재시도: 다양성 강화 문구를 추가한 힌트로 다시 호출
        out = day_chain.run({
            "context": context + "\n\n[추가 힌트] 한 끼의 후보3개는 서로 다른 주재료 가족을 반드시 사용하세요.",
            "query": user_query,
            "date": d,
            "daily_target_kcal": daily_target_kcal
        }).strip()
        out = sanitize_daily(out)

    daily_outputs.append(out)

# 5) 주간 요약 1줄 생성
week_text = "\n\n".join(daily_outputs)
sum_chain = LLMChain(llm=llm_sum, prompt=week_template)
summary_line = sum_chain.run({"weekly_text": week_text}).strip()

# 6) 최종 출력
final_text = week_text + "\n\n주간 이유/요약: " + summary_line
print(final_text)

< 2025-10-17 식단 추천 >

아침  
후보1 : 달걀찜 + 달걀말이 + 호박죽 (총 319 kcal)  
후보2 : 스크램블드에그 + 달걀찜 + 물냉면 (총 286 kcal)  
후보3 : 달걀프라이 + 호박죽 + 동치미 (총 276 kcal)  

점심  
후보1 : 오징어덮밥 + 모듬회덮밥 + 양배추샐러드 (총 422 kcal)  
후보2 : 육회비빔밥 + 잡곡밥 + 양상추샐러드 (총 419 kcal)  
후보3 : 잡채밥 + 돌솥비빔밥 + 양배추샐러드 (총 447 kcal)  

저녁  
후보1 : 코다리찜 + 고등어조림 + 달걀찜 (총 252 kcal)  
후보2 : 돼지갈비찜 + 오리탕 + 동태국 (총 299 kcal)  
후보3 : 닭조림 + 코다리조림 + 무짠지 (총 244 kcal)

< 2025-10-18 식단 추천 >
아침  
후보1 : 달걀찜 + 달걀말이 + 호박죽 (총 309 kcal)  
후보2 : 스크램블드에그 + 달걀찜 + 물냉면 (총 284 kcal)  
후보3 : 달걀프라이 + 달걀국 + 호박죽 (총 329 kcal)  

점심  
후보1 : 오징어덮밥 + 모듬회덮밥 + 양배추샐러드 (총 432 kcal)  
후보2 : 육회비빔밥 + 잡곡밥 + 양상추샐러드 (총 419 kcal)  
후보3 : 잡채밥 + 오징어젓 + 양배추샐러드 (총 434 kcal)  

저녁  
후보1 : 코다리찜 + 닭조림 + 동치미 (총 179 kcal)  
후보2 : 고등어조림 + 돼지갈비찜 + 무짠지 (총 231 kcal)  
후보3 : 오리탕 + 코다리조림 + 동치미 (총 194 kcal)

< 2025-10-19 식단 추천 >
아침  
후보1 : 달걀찜 + 달걀말이 + 호박죽 (총 309 kcal)  
후보2 : 스크램블드에그 + 달걀찜 + 물냉면 (총 285 kcal)  
후보3 : 달걀찜 + 달걀프라이 + 호박죽 (총 305 kcal)  

점심  
후보1 : 고등어조림 + 잡곡밥 + 양배추샐러드 (총 466 kcal)  
후보2 : 

--------------------------------   
-----------------------------

# 최종 수정 프롬프트

In [36]:
# === 최종 코드: 주간 식단 RAG 생성 (하루 중복검사+재시도, 주식 반복 허용, 평균 칼로리 계산) ===
import os, re, datetime, warnings
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from pinecone import Pinecone
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

warnings.filterwarnings("ignore")

# -----------------------------
# 0) API 키 / 인덱스 연결
# -----------------------------
load_dotenv()
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
assert OPENAI_API_KEY and PINECONE_API_KEY, ".env에 OPENAI_API_KEY / PINECONE_API_KEY를 설정하세요."

pc     = Pinecone(api_key=PINECONE_API_KEY)
oai    = OpenAI(api_key=OPENAI_API_KEY)
INDEX_NAME = "food-test1"   # 기존에 쓰던 이름 사용
NAMESPACE  = "foods-ns1"
index = pc.Index(INDEX_NAME)

In [37]:
# -----------------------------
# 1) LLM 설정
# -----------------------------
# 일일 생성용(조금 저렴/빠른 모델)
llm_day = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, max_tokens=1024, openai_api_key=OPENAI_API_KEY)
# 주간 요약용(숫자만 바꿔 끼우면 되므로 짧게)
llm_sum = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, max_tokens=200, openai_api_key=OPENAI_API_KEY)

today = datetime.date.today().strftime("%Y-%m-%d")

# -----------------------------
# 2) 사용자 쿼리
# -----------------------------
user_query = (
    "근육증진형, 고단백, 고지방, 저나트륨 식단 중심으로, 아침/점심/저녁 세 끼만 추천해줘. "
    "food_weight를 반영하여 하루 총 칼로리가 1800~2500kcal 사이에 오도록 정확히 계산해줘."
)

In [38]:
# -----------------------------
# 3) 프롬프트 템플릿 (banlist 지원 + 주식 반복 허용)
# -----------------------------
day_prompt_str = """
당신은 최고 수준의 영양사입니다. 아래 규칙과 참고 문서 내 음식만 사용하여 식단을 구성하세요.

**핵심 규칙:**
1. 출력 형식은 <출력 형식>을 정확히 따르세요. 마크다운/불필요한 문구/여분 줄/**'이유'라는 단어** 금지.
2. [칼로리 계산]: <참고 문서>의 **칼로리(100g)**와 **1인분 중량** 공식을 사용하여 각 음식의 1인분 칼로리를 합산해야 합니다.
3. [칼로리 목표]: 각 끼니의 **총 칼로리는 500 kcal ~ 900 kcal 사이**, 하루 총합은 1800 kcal ~ 2500 kcal 사이여야 합니다.
4. [주재료 다양성]: 같은 끼니의 후보 1, 2, 3은 모두 **서로 완전히 다른 메인 식재료**를 사용하며, **후보들 간에 단 하나의 음식도 겹치면 안 됩니다.**
5. [음식 조합]: 각 후보는 상식적으로 함께 먹는 **3가지 이상의 음식 조합**이어야 합니다 (밥/국/주반찬/부반찬 형태 권장).
6. [중복 금지 - 매우 중요]: 같은 날(아침/점심/저녁 포함)에는 동일 **메뉴명**을 한 번만 사용하세요. 단, **주식(밥/빵/죽/국/찌개/김치류/샐러드/나물)**은 반복 사용을 **허용**합니다.
7. [금지 목록 적용]: 아래 목록에 있는 음식은 어떤 후보에도 포함하지 마세요. 비어 있으면 무시하십시오.
   - 금지 음식: {banlist}

<참고 문서>
{context}

<사용자 요청>
{query}

<출력 형식> (이유/요약 섹션은 **없습니다.**)
< {date} 식단 추천 >
아침
후보1 : 음식A + 음식B + 음식C (총 nnn kcal)
후보2 : 음식A + 음식B + 음식C (총 nnn kcal)
후보3 : 음식A + 음식B + 음식C (총 nnn kcal)
점심
후보1 : 음식A + 음식B + 음식C (총 nnn kcal)
후보2 : 음식A + 음식B + 음식C (총 nnn kcal)
후보3 : 음식A + 음식B + 음식C (총 nnn kcal)
저녁
후보1 : 음식A + 음식B + 음식C (총 nnn kcal)
후보2 : 음식A + 음식B + 음식C (총 nnn kcal)
후보3 : 음식A + 음식B + 음식C (총 nnn kcal)
""".strip()

In [39]:
week_prompt_str = """
당신은 영양사입니다. 아래 주간 식단 추천 내용을 바탕으로 사용자 요청(근육증진, 고단백, 저지방, 저나트륨)에 맞춰 식단이 잘 구성되었는지 1~2문장으로 요약하세요.
[계산된 평균 칼로리] 부분은 그대로 두세요. 나중에 코드가 실제 숫자로 치환합니다.

<주간 식단 추천 내용>
{weekly_text}

<요약 형식>
근육 증진을 위해 고단백과 건강한 지방 위주로 구성하였으며, 나트륨을 낮게 유지했습니다. 하루 평균 총 칼로리는 [계산된 평균 칼로리]kcal 입니다.
""".strip()

day_template  = PromptTemplate(input_variables=["context", "query", "date", "banlist"], template=day_prompt_str)
week_template = PromptTemplate(input_variables=["weekly_text"], template=week_prompt_str)

In [40]:
# -----------------------------
# 4) Pinecone 검색 → context 구성 (k=200)
# -----------------------------
qv = oai.embeddings.create(model="text-embedding-3-small", input=user_query).data[0].embedding
res = index.query(vector=qv, top_k=200, namespace=NAMESPACE, include_metadata=True)
res = res.to_dict() if hasattr(res, "to_dict") else res
matches = res.get("matches", [])

# context: LLM이 계산에 쓸 수 있도록 100g kcal/1인분(g)/매크로 제공
context_lines = []
for m in matches:
    md = m.get("metadata", {}) or {}
    name = str(md.get("food_name", "")).strip()
    if not name: 
        continue
    food_weight   = md.get("food_weight", 100)           # 없으면 100g 가정
    kcal_per_100g = md.get("energy_kcal", 0)
    pro  = md.get("protein_g", 0); fat = md.get("fat_g", 0); carb = md.get("carb_g", 0)
    disp = md.get("display_tags", "")

    context_lines.append(
        f"음식명: {name} (1인분 중량: {food_weight}g), "
        f"칼로리(100g): {kcal_per_100g}kcal, 단백질: {pro}g, 지방: {fat}g, 탄수화물: {carb}g, 태그: {disp}"
    )

context = "\n".join(context_lines)

In [41]:
# -----------------------------
# 5) 파서/검사 유틸 (하루 중복검사, 주식 허용)
# -----------------------------
# 후보 라인 추출: "후보1 : A + B + C (총 600 kcal)"
CAND_LINE_RE = re.compile(r"^후보\d\s*:\s*(.+?)\s*\(총\s*([\d\.]+)\s*kcal\)", re.MULTILINE)

def parse_day_block(day_text: str):
    blocks = {"아침":[], "점심":[], "저녁":[]}
    current = None
    for line in day_text.splitlines():
        s = line.strip()
        if s == "아침": current = "아침"; continue
        if s == "점심": current = "점심"; continue
        if s == "저녁": current = "저녁"; continue
        if current:
            m = CAND_LINE_RE.match(s)
            if m:
                combo = m.group(1).strip()
                kcal  = float(m.group(2))
                blocks[current].append((combo, kcal))
    return blocks

def split_foods(combo: str):
    return [x.strip() for x in re.split(r"\+|＋", combo) if x.strip()]

# 주식(반복 허용) 판단: 필요 시 패턴 조정 가능
STAPLE_ALLOW_RE = re.compile(r"(밥|빵|죽|국|찌개|김치|나물|샐러드)$")
def is_staple(name: str) -> bool:
    n = re.sub(r"\s+", "", name.strip())
    return bool(STAPLE_ALLOW_RE.search(n))

def find_day_duplicates(day_struct):
    """
    하루 전체(아침/점심/저녁)의 모든 후보를 합쳐 '같은 음식명' 중
    주식이 아닌 항목의 중복만 검출.
    """
    seen = set()
    dups = set()
    for meal in ["아침","점심","저녁"]:
        for combo, _ in day_struct.get(meal, []):
            for item in split_foods(combo):
                if is_staple(item):   # 주식은 허용
                    continue
                if item in seen:
                    dups.add(item)
                else:
                    seen.add(item)
    return (len(dups) > 0, dups)

def day_total_kcal_from_top1(day_struct):
    total = 0.0
    for meal in ["아침","점심","저녁"]:
        cand = day_struct.get(meal, [])
        if cand:
            total += float(cand[0][1])
    return total


In [43]:
# -----------------------------
# 6) 7일 생성 루프: 중복 검사→필요 시 1회 재시도, 총칼로리 수집
# -----------------------------
dates = [(datetime.date.today() + datetime.timedelta(days=i)).strftime("%Y-%m-%d") for i in range(7)]
day_chain  = LLMChain(llm=llm_day,  prompt=day_template)
sum_chain  = LLMChain(llm=llm_sum,  prompt=week_template)

daily_outputs = []
daily_totals  = []

print("\n\n🍽️ 일주일치 식단 생성 중...")

for d in dates:
    # 1차 생성 (금지목록 없음)
    out = day_chain.run({"context": context, "query": user_query, "date": d, "banlist": ""}).strip()
    out = re.sub(r"이유\s*:\s*.*", "", out).strip()

    struct = parse_day_block(out)
    has_dup, dup_names = find_day_duplicates(struct)

    if has_dup:
        # 주식 제외한 중복만 banlist로 1회 재시도
        filtered_dups = {n for n in dup_names if not is_staple(n)}
        ban = ", ".join(sorted(filtered_dups)) if filtered_dups else ""
        if ban:
            out2 = day_chain.run({"context": context, "query": user_query, "date": d, "banlist": ban}).strip()
            out2 = re.sub(r"이유\s*:\s*.*", "", out2).strip()
            struct = parse_day_block(out2)
            out = out2  # 재시도 결과 채택

    daily_totals.append(day_total_kcal_from_top1(struct))
    daily_outputs.append(out)


# -----------------------------
# 7) 주간 요약 + 평균 칼로리(우리 코드로 정확 계산) 삽입
# -----------------------------
week_text = "\n\n".join(daily_outputs)
avg_kcal = (sum(daily_totals) / len(daily_totals)) if daily_totals else 0.0

summary_line = sum_chain.run({"weekly_text": week_text}).strip()
summary_line = re.sub(r"\[계산된 평균 칼로리\]kcal", f"{avg_kcal:.0f}kcal", summary_line)

final_text = week_text + "\n\n이유 : " + summary_line

print("\n\n========================================")
print("✅ 최종 주간 식단 추천 결과")
print("========================================")
print(final_text)



🍽️ 일주일치 식단 생성 중...


✅ 최종 주간 식단 추천 결과
< 2025-10-17 식단 추천 >
아침
후보1 : 고등어구이 (200g) + 현미밥 (230g) + 오이무침 (80g) (총 752 kcal)  
후보2 : 간장양념닭다리구이 (300g) + 검정콩밥 (200g) + 숙주나물 (50g) (총 883 kcal)  
후보3 : 삼겹살구이 (200g) + 기장밥 (200g) + 감자조림 (50g) (총 868 kcal)  

점심
후보1 : 오징어채볶음 (20g) + 잡채밥 (550g) + 열무나물 (90g) (총 781 kcal)  
후보2 : 제육볶음 (250g) + 흑미밥 (200g) + 미역줄기볶음 (80g) (총 842 kcal)  
후보3 : 오징어덮밥 (360g) + 잡곡밥 (200g) + 고추장아찌 (60g) (총 860 kcal)  

저녁
후보1 : 돼지고기 수육 (300g) + 달걀국 (400g) + 김치전골 (270g) (총 835 kcal)  
후보2 : 닭구이 (340g) + 오징어국 (500g) + 미역초무침 (50g) (총 792 kcal)  
후보3 : 양념갈치젓 (90g) + 오징어채조림 (50g) + 감자전 (200g) (총 817 kcal)

< 2025-10-18 식단 추천 >
아침
후보1 : 고등어구이 (200g) + 흑미밥 (200g) + 미역국 (400g) (총 575 kcal)  
후보2 : 오징어채볶음 (20g) + 검정콩밥 (200g) + 호박죽 (400g) (총 617 kcal)  
후보3 : 간장양념닭다리구이 (300g) + 현미밥 (230g) + 달걀국 (400g) (총 765 kcal)  

점심
후보1 : 제육볶음 (250g) + 영양돌솥밥 (350g) + 미역냉국 (350g) (총 872 kcal)  
후보2 : 닭볶음 (300g) + 잡채밥 (550g) + 열무나물 (90g) (총 910 kcal)  
후보3 : 간장양념치킨 (200g) + 볶음밥 (350g) + 오징어국 (500g